# **USGS DATA RETRIEVAL Model**
This model is designed to be a simplified (conceptual) model of the National Water Model, which is functionally equivalent. This is an example run of the Python implimentation of the model.
## bmi_usgs
This is the code for the Basic Model Interface (BMI) that is used to call the usgs and interact with other models via the Framework, or driving code. This code contains all the required BMI functions to run the CFE, including 
 - initialize: Perform startup tasks for the model.
 - update: Advance model state by one time step. Calls the function `run_usgs` from `usgs.py`
 - update_until: Advance model state until the given time.
 - finalize: Perform tear-down tasks for the model.
 - get_value: Get a copy of values of a given variable.
 - set_value: Set the values of a given variable.
 - etc.  
 These functions need to be called by a framework or driving code, an example of which is below.

In [1]:
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import bmi_usgs

### Create an instance of the model with a specific configuration that corresponds to a particular catchmenmt.

In [2]:
usgs_instance = bmi_usgs.BMI_USGS()

In [3]:
usgs_instance.initialize(cfg_file='./usgs_config.json', current_time_step=1)

### We will want to visualize the model output

In [4]:
outputs=usgs_instance.get_input_var_names()
output_lists = {output:[] for output in outputs}
print(output_lists)

{'sites': [], 'start': [], 'end': [], 'service': []}


In [5]:
# usgs_instance.set_value('sites','01054500')
# usgs_instance.set_value('service','peaks')
# usgs_instance.set_value('start','1970-01-01')
# usgs_instance.set_value('end','1990-01-01')

In [6]:
print(usgs_instance.get_value('sites'))
print(usgs_instance.get_value('service'))
print(usgs_instance.get_value('start'))
print(usgs_instance.get_value('end'))

0
0
0
0


In [7]:
usgs_instance

In [8]:
#run once
usgs_instance.update()

                         Date    Flow
0   2019-06-01 04:00:00+00:00  5.1975
1   2019-06-01 05:00:00+00:00  5.1825
2   2019-06-01 06:00:00+00:00  5.1625
3   2019-06-01 07:00:00+00:00  5.1575
4   2019-06-01 08:00:00+00:00  5.1500
..                        ...     ...
739 2019-07-01 23:00:00+00:00  4.5350
740 2019-07-02 00:00:00+00:00  4.5350
741 2019-07-02 01:00:00+00:00  4.5425
742 2019-07-02 02:00:00+00:00  4.5350
743 2019-07-02 03:00:00+00:00  4.5375

[744 rows x 2 columns]


In [9]:
usgs_instance.finalize(print_flow=True)

TypeError: finalize() got an unexpected keyword argument 'print_flow'

In [13]:
# plt(Date, Flow)

### **Test codes**

In [10]:
import dataretrieval.nwis as nwis
from dataretrieval import nwis, utils, codes
import pandas as pd
import matplotlib.pyplot as plt


# class USGS():
#     def __init__(self):
#         super(USGS, self).__init__()

#     # RUNNING USGS SCRIPT
#     def run_usgs(self, k):
#         # site, start='2019-02-01', end='2019-04-01', service='iv'
#         site=k.sites; service=k.service; start=k.service; end=k.service
                
#         # get peak flow # get instantaneous values (iv)
#         station_record = nwis.get_record(site=k.sites, service=k.service, start=k.start, end=k.end)
#         print(station_record['peak_va'])
#         # station_record = station_record['peak_va'] #.rename(['Flow'])
#         # # station_record = station_record.resample('H').mean()

        
#         return 

    
    
class USGS():
    def __init__(self):
        super(USGS, self).__init__()
        
    # __________________________________________________________________________________________________________
    # RUNNING USGS SCRIPT
    def run_usgs(self, k):
        
        # get instantaneous values (iv)
        site=k.sites; service=k.service; start=k.service; end=k.service
                
        # get peak flow # get instantaneous values (iv)
        station_record = nwis.get_record(site=k.sites, service=k.service, start=k.start, end=k.end)
        
        station_record.reset_index(inplace=True)
        station_record['datetime'] = pd.to_datetime(station_record['datetime'], utc=True, format = '%Y-%m-%d %H:%M:%S') 
        station_record_quarter = station_record.iloc[:,[0,4]]
        station_record_quarter.columns = ['Date', 'Flow']
        
        ## Make copy od dataframe to average flow every hour 
        station_record_copy = station_record.copy()
        station_record_copy['datetime'] = pd.to_datetime(station_record_copy['datetime'], utc=True, format = '%Y-%m-%d %H:%M:%S')
        station_record_copy.index = station_record_copy['datetime']
        station_record_avg = station_record_copy.resample('H').mean()
        station_record_avg.reset_index(inplace=True)
        
        station_record_avgflow = station_record_avg.iloc[:,[2]] #station_record_avg.iloc[:,[0,2]]
        station_record_avgflow.columns = ['Flow']  #['Date', 'Flow']
        print(station_record_avgflow)
        
        return    
    
    
    
    
    

In [11]:
sites='03339000'
service='iv'
start='1970-01-01'
end='1990-01-01'

In [12]:
USGS.run_usgs(sites,service,start,end)

TypeError: run_usgs() takes 2 positional arguments but 4 were given